In [1]:
from bs4 import BeautifulSoup
from PttWebCrawler.crawler import PttWebCrawler
from dateutil import parser
from datetime import datetime, timedelta

import requests
import re
import time
import json

In [2]:
VERIFY = True
PTT_URL = "https://www.ptt.cc"

In [15]:
def _parse_articles(index, board, timeout=3):
    link_aid_board_set = set()
    resp = requests.get(
        url=PTT_URL + "/bbs/" + board + "/index" + str(index) + ".html",
        cookies={"over18": "1"},
        verify=VERIFY,
        timeout=timeout,
    )
    if resp.status_code != 200:
        print("invalid url:", resp.url)
        return []

    soup = BeautifulSoup(resp.text, "html.parser")
    divs = soup.find_all("div", "r-ent")
    for div in divs:
        try:
            # ex. link would be <a href="/bbs/PublicServan/M.1127742013.A.240.html">Re: [問題] 職等</a>
            href = div.find("a")["href"]
            link = PTT_URL + href
            article_id = re.sub("\.html", "", href.split("/")[-1])
            link_aid_board_set.add((link, article_id, board))
        except:
            pass
    return list(link_aid_board_set)


def _gen_es_article(article, idx_name, board, link):
    def comment_ipdatetime_2_datetime(ipdatetime:str, year:str)->datetime:
        ipdatetime = ipdatetime.split(" ")
        if len(ipdatetime) == 3:
            dt = parser.parse(f"{year}/{ipdatetime[1]} {ipdatetime[2]}", ignoretz=True)
            return dt.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        if len(ipdatetime) == 2:
            dt = parser.parse(f"{year}/{ipdatetime[0]} {ipdatetime[1]}", ignoretz=True)
            return dt.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        return None

    date = article["date"].strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    es_article = {
        "_op_type": "index",
        "_index": idx_name,
        "status_code": "UN_ASTE",
        "platform": "PTT",
        "borad": board,
        "link": link,
        "date": date,
        "crawl_time": crawl_time,
        "title": article["article_title"],
        "context": article["content"],
        "comments": [
            {
                "username": message["push_userid"],
                "content": message["push_content"],
                "date": comment_ipdatetime_2_datetime(message["push_ipdatetime"], date[:4]),
            }
            for message in article["messages"]
            if message["push_content"]
        ],
    }
    return es_article


def get_ptt_article_generator(
    idx_name: str, board="Gossiping", least_n_days: float = 7
):
    least_n_days = timedelta(days=least_n_days)
    last_page = PttWebCrawler.getLastPage(board)
    page_pointer = last_page
    now = datetime.now()
    while page_pointer > 0:
        link_aid_board_list = _parse_articles(page_pointer, board)
        for link, aid, board in link_aid_board_list:
            try:
                article_str = PttWebCrawler.parse(link, aid, board)
                article = json.loads(article_str)
                article["date"] = parser.parse(
                    article["date"], ignoretz=True
                ) - timedelta(hours=8)
            except Exception as e:
                page_pointer = 0
                break
            if (now - article["date"]) > least_n_days:
                if page_pointer == last_page:
                    continue
                else:
                    page_pointer = 0
                    break
            else:
                yield _gen_es_article(article, idx_name, board, link)
        page_pointer -= 1

In [16]:
list(get_ptt_article_generator("ptt", "Stock", 0.1))

Processing article: M.1730916378.A.2C0
Processing article: M.1730915738.A.460
Processing article: M.1730910698.A.7C3
Processing article: M.1730914841.A.7A3
Processing article: M.1730911748.A.E0E
Processing article: M.1730913210.A.170
Processing article: M.1730910326.A.DEF
Processing article: M.1730872802.A.680
Processing article: M.1730738309.A.238
Processing article: M.1730911261.A.135
Processing article: M.1730912786.A.751
Processing article: M.1719872231.A.9BA
Processing article: M.1730906726.A.069


[{'_op_type': 'index',
  '_index': 'ptt',
  'status_code': 'UN_ASTE',
  'platform': 'PTT',
  'borad': 'Stock',
  'link': 'https://www.ptt.cc/bbs/Stock/M.1730916378.A.2C0.html',
  'date': '2024-11-06 18:06:15.000',
  'crawl_time': '2024-11-07 02:55:52.301',
  'title': 'Re: [新聞] 川普才剛當選！傳德州州長將來台 要台積',
  'context': '我只能說川普效應大概就4年，2nm台灣才剛蓋7座場，光是蓋場效率就抵不過晶圓需求了， 等蓋到完，總統又換人了 台積電於2021年在美國亞利桑那州鳳凰城開始建設第一座晶圓廠，原計劃於2024年量產4奈 米製程，但由於人力短缺和建設成本上升等因素，量產時間延後至2025年。 以這個類比，川皇上台時動土剪綵德州場，下台時第一片晶圓不知道孵出來了沒 : 標題: Re: [新聞] 川普才剛當選傳德州州長將來台 要台積 : 時間: Thu Nov 7 00:25:24 2024 : : 如題 : : 之前就講過了 : 台積電騰籠換鳥 愈做愈強 : 台灣島籠中無鳥 愈來愈慘 : : 台積電未來20年肯定還是頂尖公司 : 只是台灣就不一定了 : 台積電裡面幹的事情就是製造業 : 製造業只有有人有錢就搞的起來 : 你真的覺得美國有真的缺錢還是缺人 : 政治力量是可以壓倒一切的 : : 再說了很多事情都是有跡可循 : 之前的新聞 : 為什麼一間世界市值前十的公司 : 要給日本高中生學費跟生活費補助 : 讓他們來台灣念大學 : 你能想像英特爾極盛時期 : 補助台灣高中生去美國讀大學嗎 : : 亮哥之前提出來還被狂噓 保護費不一定是錢 : 美國日本打的算盤局外人應該都看的清楚了 : : : 我已經清晰可以預見20年後的2045 : 2030年出生的台灣人問長輩 : Morris是誰 為什麼當初可以國葬 : 長輩答道 他呀 : 給台灣人創造了一家本不屬於這座島的公司 : 那是一個念理工科系就有一份體面薪水的時代 : 那是一個台灣人懷揣夢想前進股市的時代 : 那是一個最好的時代

In [ ]:
{
    "_op_type": "index",
    "_index": idx_name,
    "status_code": "UN_ASTE",
    "platform": "PTT",
    "borad": board,
    "link": i["link"],
    "date": i["date"].replace("T", " ").replace("Z", ""),
    "crawl_time": i["crawl_time"].replace("T", " ").replace("Z", ""),
    "title": i["title"],
    "context": i["context"],
    "comments": [
        {
            "username": j[0],
            "content": j[1],
            "date": j[2].replace("T", " ").replace("Z", ""),
        }
        for j in i["comments"]
        if j[1]
    ],
}